In [2]:
%matplotlib inline  
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets, neighbors , preprocessing ,tree


In [3]:
kanser=datasets.load_breast_cancer()
print(kanser.DESCR)

Breast Cancer Wisconsin (Diagnostic) Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, field
        13 is Radius SE, field 23 is Worst Radius.

        

In [ ]:
Verileri böyle görünce kafanızda pek bir şey canlanmamıştır diye düşünüyorum. Bu verileri dataframe olarak gösterelim.
Bunun için Pandas kütüphanesinin DataFrame() fonksiyonunu kullanacağız ve ardından head() fonksiyonu ile
ilk 5 veriyi görüntüleyeceğiz.

In [12]:
raw_data=kanser.data
#print(raw_data)

kanser_df=pd.DataFrame(raw_data)
kanser_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
Sanırım şimdi daha açıklayıcı oldu ancak 0,1,2,3… diye giden parametrelerin ne anlama geldiğini bilmiyoruz.
Bunun için oluşturduğumuz dataframe’in columns parametresini güncelleyelim.

In [5]:
kanser_df.columns = kanser.feature_names
kanser_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [6]:
output_df = pd.DataFrame(kanser.target, columns=["Çıktı"])
output_df.head()

,Çıktı
0,0
1,0
2,0
3,0
4,0


In [ ]:
"Evet verileri düzgün bir şekilde görebiliyoruz, 
son olarak bütün parametrelerin ve çıktıların bir arada olduğu genel bir dataframe oluşturalım. 
Bunun için Pandas kütüphanesinin concat() fonksiyonunu kullanacağız,
ancak çıktıların en sonda kalmasını istediğimiz için sort parametresine False değerini veriyoruz."

In [7]:
all_df = pd.concat([kanser_df, output_df], sort=False, axis=1)
all_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Çıktı
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [ ]:
# iris = datasets.load_iris()
# X = iris.data[:, :2]  # ilk 2 sütunu eğitime almak için
# y = iris.target

In [ ]:
Olayı anladığımıza göre artık verileri eğitim ve test verisi olarak ikiye bölebiliriz 
fakat bundan önce hangi parametrede eğitim yapmak istediğimizi seçmeliyiz.
Bu eğitim için “bmi” yani vücut kitle indeksini kullanacağım fakat siz hepsini veya
başka bir parametreyi kullanarak eğitim yapabilirsiniz.

In [ ]:
# X = kanser_df['mean radius'].values
# y = output_df.values
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=9)
# X_train = X_train.reshape(-1,1)
# X_test= X_test.reshape(-1,1)

In [ ]:
Scikit-learn kütüphanesiden train_test_split() fonksiyonu yardımıyla,
%80’i eğitim ve %20’si test verisi olacak şekilde x_train, x_test, y_train ve y_test değişkenlerine atadık.
Parametrelerde bulunan random_state = 9‘un amacı ise
siz bu fonksiyonu çağırdığınızda aynı sonuçları alabilmeniz için kullandığımız bir yöntem yani zorunlu değil.
Ardından X_train ve X_test verilerinin eğitim sırasında 
problem oluşturmaması için reshape() fonksiyonuyla boyutlarını düzenledik.